In [1]:
# LIBRARIES
import snowflake.connector
import logging
from os import environ
from dotenv import load_dotenv
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
import optuna
import joblib
import numpy as np

In [2]:
# SNOWFLAKE CONNECTION 
load_dotenv()
for logger_name in ['snowflake.connector', 'botocore']:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.DEBUG)
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    logger.addHandler(handler)

conn = snowflake.connector.connect(
    user = environ['USER'],
    password = environ['PASSWORD'],
    account = environ['ACCOUNT'],
    warehouse = environ['WAREHOUSE'],
    database = environ['DATABASE'],
    schema = environ['SCHEMA']
)

Snowflake Connector for Python Version: 3.12.2, Python Version: 3.10.14, Platform: Windows-10-10.0.22631-SP0
connect
__config
Connecting to GLOBAL Snowflake domain
This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
use_numpy: False
initialized
REST API object was created: fa62087.us-east-2.aws.snowflakecomputing.com:443
authenticate
assertion content: *********
account=fa62087, user=PAULO_MAIA, database=POKER_PROD, schema=POKER, warehouse=REPORTING_POKER_WH, role=None, request_id=02d264df-48f3-4b84-8e20-77e0f07136ab
body['data']: {'CLIENT_APP_ID': 'PythonConnector', 'CLIENT_APP_VERSION': '3.12.2', 'SVN_REVISION': None, 'ACCOUNT_NAME': 'fa62087', 'LOGIN_NAME': 'PAULO_MAIA', 'CLIENT_ENVIRONMENT': {'APPLICATION': 'PythonConnector', 'OS': 'Windows', 'OS_VERSION': 'Windows-10-10.0.22631-SP0', 'PYTHON_VERSI

In [3]:
# SELECTING THE QUERY AND GETTING THE DATA
with open('training_query.sql', 'r') as file:
    query = file.read()
    
data = pd.read_sql_query(query, conn)

C:\Users\maiap\AppData\Local\Temp\ipykernel_30568\672369880.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)
cursor
executing SQL/command
query: [WITH counts AS ( SELECT start_date_local, start_hour_local, COUNT(*) AS same_hou...]
binding: [WITH counts AS ( SELECT start_date_local, start_hour_local, COUNT(*) AS same_hou...] with input=[None], processed=[{}]
sequence counter: 1
Request id: 5f22852d-8eff-44bf-a9b0-b52af04be848
running query [WITH counts AS ( SELECT start_date_local, start_hour_local, COUNT(*) AS same_hou...]
is_file_transfer: True
_cmd_query
serialize_to_dict() called
sql=[WITH counts AS ( SELECT start_date_local, start_hour_local, COUNT(*) AS same_hou...], sequence_id=[1], is_file_transfer=[False]
Opentelemtry otel injection failed because of: No module named 'opentelemetry'


In [5]:
# Converter para o formato de data
data['START_DATE_LOCAL'] = pd.to_datetime(data['START_DATE_LOCAL'])
# Definir o tamanho das janelas
training_window = pd.DateOffset(months=11)
validation_window = pd.DateOffset(months=1)

# Criar o loop das janelas
start_date = pd.to_datetime('2023-07-30')  # Primeira janela de validação
end_date = (pd.Timestamp.now() - pd.DateOffset(days=1)).date()
end_date = pd.Timestamp(end_date)  # Converte para Timestamp# Última janela de validação
current_date = start_date


In [ ]:
# ENCODING CATEGORICAL COLUMNS AND DEFINING X AND Y
encoder = OneHotEncoder(sparse_output=False)
data_transform = data.drop(columns=['MATCH_ID'])
cat_columns = ['START_HOUR_LOCAL', 'DAY_OF_WEEK_LOCAL', 'KO_TYPE', 
               'MTT_POOL_ALLOCATION_DESCRIPTION', 'ESTRUTURA_BLINDS']
encoded_columns = encoder.fit_transform(data_transform[cat_columns])
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(cat_columns))
data_final = pd.concat([data_transform.drop(cat_columns, axis=1), encoded_df], axis=1)

X = data_final.drop(columns=['HAS_OVERLAY', 'OVERLAY', 'MTT_RESULT', 'COLLECTED', 'NORMALIZED_MTT_RESULT', 'NORMALIZED_COLLECTED' ], axis=1)
y = data_final['COLLECTED']

In [34]:
while current_date <= end_date:
    # Definir os períodos de treinamento e validação
    training_start = current_date - training_window
    training_end = current_date - pd.Timedelta(days=1)  # Um dia antes da validação
    validation_start = current_date
    validation_end = current_date + validation_window - pd.Timedelta(days=1)  # Último dia do mês

    # Filtrar os dados para treinamento e validação
    training_data = data_final[(data_final['START_DATE_LOCAL'] >= training_start) & (data_final['START_DATE_LOCAL'] <= training_end)]
    validation_data = data_final[(data_final['START_DATE_LOCAL'] >= validation_start) & (data_final['START_DATE_LOCAL'] <= validation_end)]

    # Aqui você pode treinar e validar o modelo
    print(f"Treinamento: {training_start.date()} a {training_end.date()}")
    print(f"Validação: {validation_start.date()} a {validation_end.date()}\n")

    # Avançar a janela para o próximo mês
    current_date += pd.DateOffset(months=1)

Treinamento: 2022-08-30 a 2023-07-29
Validação: 2023-07-30 a 2023-08-29

Treinamento: 2022-09-30 a 2023-08-29
Validação: 2023-08-30 a 2023-09-29

Treinamento: 2022-10-30 a 2023-09-29
Validação: 2023-09-30 a 2023-10-29

Treinamento: 2022-11-30 a 2023-10-29
Validação: 2023-10-30 a 2023-11-29

Treinamento: 2022-12-30 a 2023-11-29
Validação: 2023-11-30 a 2023-12-29

Treinamento: 2023-01-30 a 2023-12-29
Validação: 2023-12-30 a 2024-01-29

Treinamento: 2023-02-28 a 2024-01-29
Validação: 2024-01-30 a 2024-02-28

Treinamento: 2023-03-29 a 2024-02-28
Validação: 2024-02-29 a 2024-03-28

Treinamento: 2023-04-29 a 2024-03-28
Validação: 2024-03-29 a 2024-04-28

Treinamento: 2023-05-29 a 2024-04-28
Validação: 2024-04-29 a 2024-05-28

Treinamento: 2023-06-29 a 2024-05-28
Validação: 2024-05-29 a 2024-06-28

Treinamento: 2023-07-29 a 2024-06-28
Validação: 2024-06-29 a 2024-07-28

Treinamento: 2023-08-29 a 2024-07-28
Validação: 2024-07-29 a 2024-08-28

Treinamento: 2023-09-29 a 2024-08-28
Validação: 202

In [6]:
# DEFINING THE CROSS-VALIDATION FUNCTION FOR XGBOOST REGRESSOR MODEL WITH OPTUNA
def cross_validation_optuna(trial, X, y, n_splits):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.8)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0, 5)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-5, 1e1, log=True)
    reg_lambda = trial.suggest_float('reg_lambda', 1e-5, 1e1, log=True)


    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma = gamma,
        min_child_weight = min_child_weight,
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        device='cuda',
        random_state=42
    )

    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    results_list = []

    for train_idx, test_idx in kfold.split(X):
        X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        # CREATION OF A DATAFRAME TO SAVE ALL THE RESULTS
        results_df = pd.DataFrame({
            'Actual': y_test,
            'Prediction': predictions,
        })

        # ADDING ADDITIONAL METRICS TO THE DATAFRAME
        results_df['Absolute difference'] = np.abs(results_df['Actual'] - results_df['Prediction'])
        results_df['Mean square difference'] = (results_df['Actual'] - results_df['Prediction']) ** 2
        results_df['Error percentage'] = (results_df['Actual'] - results_df['Prediction']) / results_df['Actual'] * 100
        results_df['Absolute error percentage'] = np.abs(results_df['Error percentage'])

        results_list.append(results_df)

    full_results_df = pd.concat(results_list, ignore_index=True)

    return full_results_df, model


# DEFINING THE OPTIMIZATION FUNCTION FOR HYPERPARAMETER TUNING
def optimize_hyperparameters(X, y):
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())

    def objective(trial):
        full_results_df, model = cross_validation_optuna(trial, X, y, n_splits=10)
        trial.set_user_attr("model", model)
        trial.set_user_attr("results", full_results_df)
        return full_results_df['Absolute difference'].median()  # Using MSE as the optimization target

    study.optimize(objective, n_trials=40)

    print("Best hyperparameters: ", study.best_params)

    best_trial = study.best_trial
    best_model = best_trial.user_attrs["model"]
    best_results = best_trial.user_attrs["results"]

    return best_model, best_results

# RUNNING THE MODEL 
best_model, best_results_df = optimize_hyperparameters(X, y)

[I 2024-11-19 16:08:08,266] A new study created in memory with name: no-name-47dfe1bd-2198-4035-bc20-1fed715fb3c0
C:\Users\maiap\anaconda3\envs\suprema-env\lib\site-packages\xgboost\core.py:158: UserWarning: [16:08:09] WARNING: D:\bld\xgboost-split_1727635012356\work\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2024-11-19 16:08:15,937] Trial 0 finished with value: 35.31689453125 and parameters: {'n_estimators': 146, 'max_depth': 8, 'learning_rate': 0.46392456008558997, 'subsample': 0.8295531185538675, 'colsample_bytree': 0.5115167272086848, 'gamma': 0.043116108372410755, 'min_child_we

Best hyperparameters:  {'n_estimators': 110, 'max_depth': 16, 'learning_rate': 0.06816448127304066, 'subsample': 0.9112424040826942, 'colsample_bytree': 0.6819324010072926, 'gamma': 0.016392012530653144, 'min_child_weight': 6, 'reg_alpha': 0.0007971807194501131, 'reg_lambda': 0.02278351502837931}


In [7]:
# SAVING THE MODEL AND THE ENCODER
joblib.dump(encoder, '../new_model/history_encoder.pkl')
joblib.dump(best_model, '../new_model/history_model.pkl')

['../new_model/history_model.pkl']

In [36]:
# ENCODING CATEGORICAL COLUMNS AND DEFINING X AND Y
encoder = OneHotEncoder(sparse_output=False)
data_transform = data.drop(columns=['MATCH_ID', 'START_DATE_LOCAL'])
cat_columns = ['START_HOUR_LOCAL', 'DAY_OF_WEEK_LOCAL', 'KO_TYPE', 
               'MTT_POOL_ALLOCATION_DESCRIPTION', 'ESTRUTURA_BLINDS']
encoded_columns = encoder.fit_transform(data_transform[cat_columns])
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(cat_columns))
data_final = pd.concat([data_transform.drop(cat_columns, axis=1), encoded_df], axis=1)

X = data_final.drop(columns=['HAS_OVERLAY', 'OVERLAY', 'MTT_RESULT', 'COLLECTED', 'NORMALIZED_MTT_RESULT', 'NORMALIZED_COLLECTED' ], axis=1)
y = data_final['COLLECTED']

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# FUNÇÃO DE VALIDAÇÃO CRUZADA PARA XGBOOST COM JANELA DESLIZANTE
def cross_validation_optuna_time_series(trial, X, y, train_months, test_months):
    # Sugerindo hiperparâmetros
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.8)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0, 5)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-5, 1e1, log=True)
    reg_lambda = trial.suggest_float('reg_lambda', 1e-5, 1e1, log=True)

    # Inicializando o modelo
    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        device='cuda',
        random_state=42
    )

    # Configurando a validação temporal com janela deslizante
    tscv = TimeSeriesSplit(gap=0, test_size=test_months, train_size=train_months)
    results_list = []

    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        # Criando um DataFrame para salvar os resultados
        results_df = pd.DataFrame({
            'Actual': y_test,
            'Prediction': predictions,
        })

        # Adicionando métricas ao DataFrame
        results_df['Absolute difference'] = np.abs(results_df['Actual'] - results_df['Prediction'])
        results_df['Mean square difference'] = (results_df['Actual'] - results_df['Prediction']) ** 2
        results_df['Error percentage'] = (results_df['Actual'] - results_df['Prediction']) / results_df['Actual'] * 100
        results_df['Absolute error percentage'] = np.abs(results_df['Error percentage'])

        results_list.append(results_df)

    # Consolidando os resultados
    full_results_df = pd.concat(results_list, ignore_index=True)

    return full_results_df, model

# ALTERANDO A FUNÇÃO DE OTIMIZAÇÃO PARA UTILIZAR A VALIDAÇÃO TEMPORAL
def optimize_hyperparameters_time_series(X, y):
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())

    def objective(trial):
        # Usando 11 meses para treinamento e 1 mês para validação
        full_results_df, model = cross_validation_optuna_time_series(trial, X, y, train_months=11, test_months=1)
        trial.set_user_attr("model", model)
        trial.set_user_attr("results", full_results_df)
        return full_results_df['Absolute difference'].median()  # Usando o erro absoluto mediano como métrica de otimização

    study.optimize(objective, n_trials=40)

    print("Best hyperparameters: ", study.best_params)

    best_trial = study.best_trial
    best_model = best_trial.user_attrs["model"]
    best_results = best_trial.user_attrs["results"]

    return best_model, best_results

# EXECUTANDO A OTIMIZAÇÃO
best_model, best_results_df = optimize_hyperparameters_time_series(X, y)


In [6]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
import optuna
import pickle

# Encoding das colunas categóricas
encoder = OneHotEncoder(sparse_output=False)
data_transform = data.drop(columns=['MATCH_ID'])
cat_columns = ['START_HOUR_LOCAL', 'DAY_OF_WEEK_LOCAL', 'KO_TYPE', 
               'MTT_POOL_ALLOCATION_DESCRIPTION', 'ESTRUTURA_BLINDS']
encoded_columns = encoder.fit_transform(data_transform[cat_columns])
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(cat_columns))
data_final = pd.concat([data_transform.drop(cat_columns, axis=1), encoded_df], axis=1)

# Preparar X e y
X = data_final.drop(columns=['HAS_OVERLAY', 'OVERLAY', 'MTT_RESULT', 'COLLECTED', 'NORMALIZED_MTT_RESULT', 
                             'NORMALIZED_COLLECTED', 'START_DATE_LOCAL'], axis=1)  # Remove a coluna de data aqui
y = data_final['COLLECTED']

# Função para otimizar hiperparâmetros
def optimize_hyperparameters(X_train, y_train, period_label):
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())

    def objective(trial):
        # Hiperparâmetros
        n_estimators = trial.suggest_int('n_estimators', 50, 200)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.8)
        subsample = trial.suggest_float('subsample', 0.5, 1.0)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
        gamma = trial.suggest_float('gamma', 0, 5)
        min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
        reg_alpha = trial.suggest_float('reg_alpha', 1e-5, 1e1, log=True)
        reg_lambda = trial.suggest_float('reg_lambda', 1e-5, 1e1, log=True)

        model = xgb.XGBRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            gamma=gamma,
            min_child_weight=min_child_weight,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            device='cuda',
            random_state=42
        )

        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = []

        for train_idx, test_idx in kfold.split(X_train):
            X_train_fold, X_test_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
            y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]

            model.fit(X_train_fold, y_train_fold)
            predictions = model.predict(X_test_fold)

            # Mean Absolute Error como métrica
            mae = np.mean(np.abs(predictions - y_test_fold))
            scores.append(mae)

        return np.mean(scores)

    # Rodar a otimização
    study.optimize(objective, n_trials=40)

    # Treinar modelo final com os melhores hiperparâmetros
    best_params = study.best_params
    final_model = xgb.XGBRegressor(**best_params, device='cuda', random_state=42)
    final_model.fit(X_train, y_train)

    # Salvar o modelo
    filename = f"xgb_model_{period_label}.pkl"
    with open(filename, "wb") as file:
        pickle.dump(final_model, file)
    print(f"Modelo salvo como: {filename}")

    return final_model

# Loop temporal
data_final['START_DATE_LOCAL'] = pd.to_datetime(data_final['START_DATE_LOCAL'])
training_window = pd.DateOffset(months=11)
validation_window = pd.DateOffset(months=1)
start_date = pd.to_datetime('2023-07-30')
end_date = pd.Timestamp(pd.Timestamp.now() - pd.DateOffset(days=1))

current_date = start_date

while current_date <= end_date:
    # Definir janelas de treinamento e validação
    training_start = current_date - training_window
    training_end = current_date - pd.Timedelta(days=1)
    validation_start = current_date
    validation_end = current_date + validation_window - pd.Timedelta(days=1)

    training_data = data_final[(data_final['START_DATE_LOCAL'] >= training_start) & (data_final['START_DATE_LOCAL'] <= training_end)]
    validation_data = data_final[(data_final['START_DATE_LOCAL'] >= validation_start) & (data_final['START_DATE_LOCAL'] <= validation_end)]

    X_train = training_data.drop(columns=['COLLECTED', 'START_DATE_LOCAL'])  # Certifique-se de excluir a coluna de data
    y_train = training_data['COLLECTED']

    X_valid = validation_data.drop(columns=['COLLECTED', 'START_DATE_LOCAL'])
    y_valid = validation_data['COLLECTED']

    # Nome do período atual
    period_label = f"{training_start.date()}_to_{validation_end.date()}"

    # Otimizar os hiperparâmetros para a janela atual
    print(f"Tunando hiperparâmetros para período de {training_start.date()} a {validation_end.date()}")
    best_model = optimize_hyperparameters(X_train, y_train, period_label)

    # Avançar janela
    current_date += pd.DateOffset(months=1)


[I 2024-11-29 12:15:48,186] A new study created in memory with name: no-name-294d6558-4806-4d22-9b2e-590b73c2a5d7


Tunando hiperparâmetros para período de 2022-08-30 a 2023-08-29


C:\Users\maiap\anaconda3\envs\suprema-env\lib\site-packages\xgboost\core.py:158: UserWarning: [12:15:49] WARNING: D:\bld\xgboost-split_1727635012356\work\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2024-11-29 12:15:52,761] Trial 0 finished with value: 21.579745441040927 and parameters: {'n_estimators': 198, 'max_depth': 7, 'learning_rate': 0.5463167860909405, 'subsample': 0.8100166315977635, 'colsample_bytree': 0.8798264524073864, 'gamma': 1.869642262935295, 'min_child_weight': 3, 'reg_alpha': 0.00034043266433635773, 'reg_lambda': 0.001020651318136569}. Best is trial 0 with value: 21

Modelo salvo como: xgb_model_2022-08-30_to_2023-08-29.pkl
Tunando hiperparâmetros para período de 2022-09-30 a 2023-09-29


[I 2024-11-29 12:21:00,211] Trial 0 finished with value: 31.59114790902775 and parameters: {'n_estimators': 192, 'max_depth': 20, 'learning_rate': 0.3195678088389851, 'subsample': 0.6464378675319098, 'colsample_bytree': 0.6557641931435344, 'gamma': 2.041217415887196, 'min_child_weight': 4, 'reg_alpha': 0.00242096289708799, 'reg_lambda': 0.8021680681438014}. Best is trial 0 with value: 31.59114790902775.
[I 2024-11-29 12:21:06,223] Trial 1 finished with value: 60.099846614121006 and parameters: {'n_estimators': 141, 'max_depth': 18, 'learning_rate': 0.7110962872657661, 'subsample': 0.5202260485952838, 'colsample_bytree': 0.5563596723021348, 'gamma': 2.796931632450046, 'min_child_weight': 10, 'reg_alpha': 0.00046177251330000184, 'reg_lambda': 0.004195673141025066}. Best is trial 0 with value: 31.59114790902775.
[I 2024-11-29 12:21:08,034] Trial 2 finished with value: 32.35436895905624 and parameters: {'n_estimators': 130, 'max_depth': 4, 'learning_rate': 0.6823897256078283, 'subsample': 

...

Modelo salvo como: xgb_model_2023-09-29_to_2024-09-28.pkl
Tunando hiperparâmetros para período de 2023-10-29 a 2024-10-28


[I 2024-11-29 13:09:24,366] Trial 0 finished with value: 33.74736627803353 and parameters: {'n_estimators': 132, 'max_depth': 3, 'learning_rate': 0.08094862687530989, 'subsample': 0.9776883113485597, 'colsample_bytree': 0.9152672794584653, 'gamma': 1.1174462220786925, 'min_child_weight': 4, 'reg_alpha': 0.0005628387321111403, 'reg_lambda': 1.1333678073688274}. Best is trial 0 with value: 33.74736627803353.
[I 2024-11-29 13:09:26,333] Trial 1 finished with value: 36.6552228212195 and parameters: {'n_estimators': 127, 'max_depth': 4, 'learning_rate': 0.7704395191800797, 'subsample': 0.831474765697652, 'colsample_bytree': 0.707742224822195, 'gamma': 1.5560993716659954, 'min_child_weight': 6, 'reg_alpha': 1.2849346873192504e-05, 'reg_lambda': 6.487603156413612e-05}. Best is trial 0 with value: 33.74736627803353.
[I 2024-11-29 13:09:30,786] Trial 2 finished with value: 27.194870658155754 and parameters: {'n_estimators': 121, 'max_depth': 14, 'learning_rate': 0.70605670948224, 'subsample': 0

Modelo salvo como: xgb_model_2023-10-29_to_2024-10-28.pkl
Tunando hiperparâmetros para período de 2023-11-29 a 2024-11-28


[I 2024-11-29 13:12:23,338] Trial 0 finished with value: 37.30910535087326 and parameters: {'n_estimators': 150, 'max_depth': 6, 'learning_rate': 0.6336940760017286, 'subsample': 0.6934703943392279, 'colsample_bytree': 0.6280894518407387, 'gamma': 4.102640215689577, 'min_child_weight': 3, 'reg_alpha': 0.3836851490700641, 'reg_lambda': 0.0013878046092042277}. Best is trial 0 with value: 37.30910535087326.
[I 2024-11-29 13:12:27,169] Trial 1 finished with value: 32.49660911948679 and parameters: {'n_estimators': 83, 'max_depth': 18, 'learning_rate': 0.6396707912767787, 'subsample': 0.5219064028874176, 'colsample_bytree': 0.8985648684063727, 'gamma': 2.9895143277035103, 'min_child_weight': 5, 'reg_alpha': 0.028631906141643706, 'reg_lambda': 0.0023627482092675894}. Best is trial 1 with value: 32.49660911948679.
[I 2024-11-29 13:12:30,173] Trial 2 finished with value: 64.41173195141685 and parameters: {'n_estimators': 60, 'max_depth': 18, 'learning_rate': 0.6585880110119566, 'subsample': 0.

Modelo salvo como: xgb_model_2023-11-29_to_2024-11-28.pkl
